The knowledge of GUI programming and inserting a plot into tkinter is from https://pythonprogramming.net/tkinter-depth-tutorial-making-actual-program/. And I thank valuable help from people on Stackoverflow. It's a still naive window but it's a relatively clear version to show the 1D EM wave animation. The code in the second and third cells are from Jagrit's code, you can see a full commented version in Upsilon-Wave-Team/Physics-Engine part, and the sliders' code is from Veronica, you can see the slider part under graphics folder as well.

In [1]:
import tkinter as tk
import matplotlib
matplotlib.use("TkAgg")
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation
from matplotlib.animation import ArtistAnimation
import matplotlib.pyplot as plt
import numpy as np

#Setting constants 
ep_0 = 8.8541878*(10**(-12))
mew_0 = 4*(np.pi)*(10**(-7))
c0 = 299792458

In [2]:
class Grid:
    
    def __init__(self, grid_res = 0.1, t_steps = 500, x_steps = 200, y_steps = 200):
        
        self.ds = grid_res      #Grid Resolution (size of space step)
        self.x_steps = x_steps  #No. of point on x axis
        self.y_steps = y_steps  #No. of point on y axis
        
        self.x_len  = (self.x_steps-1)*self.ds
        self.x_axis = np.linspace(0, self.x_len, self.x_steps) 
        
        self.y_len  = (self.y_steps-1)*self.ds
        self.y_axis = np.linspace(0, self.y_len, self.y_steps)
        
        self.dt = self.ds/(np.sqrt(2.0)*c0) #dt is chosen using courant stability condition
        
        self.t_steps = t_steps              #No. of steps in simulation
        self.time_period = (self.t_steps-1)*self.dt
        self.t_axis = np.linspace(0, self.time_period, self.t_steps)
        
        self.mew_r = np.ones((self.x_steps,self.y_steps)) #relative permeability at every point in the grid; set to 1 in constructor
        self.ep_r = np.ones((self.x_steps,self.y_steps))  #relative permittivity at every point in the grid; set to 1 in constructor
        self.σ = np.zeros((self.x_steps,self.y_steps)) #electriclal conductivity at every point in the grid; set to 0 in constructor
        self.n = np.sqrt(self.mew_r*self.ep_r)  # refractive index at every point in the grid
        
        self.bndry_sz = 0
        
        self.Es = np.zeros((self.t_steps, self.x_steps, self.y_steps))
    
    def set_environment_parameters(self, mew_r = 1.0, ep_r =1.0, σ = 0.0, portion = 'half Y'):
        end = self.bndry_sz + 4
        
        if portion == 'whole grid':
            self.mew_r[:,:] = mew_r  # relative permeability in whole grid
            self.ep_r[:,:] = ep_r   # relative permittivity in whole grid
            self.σ[:,:] = σ  # electrical conductivity in whole grid
            self.n = np.sqrt(self.mew_r*self.ep_r)
        
        elif portion == 'half Y':
            self.mew_r[:,self.y_steps//2:-end] = mew_r  # relative permeability in part of the grid
            self.ep_r[:,self.y_steps//2:-end]  = ep_r   # relative permittivity in part of the grid
            self.σ[:,self.y_steps//2:-end] = σ  # electrical conductivity in part of the grid
            self.n = np.sqrt(self.mew_r*self.ep_r)
        
        elif portion == 'half X':
            self.mew_r[end:self.x_steps//2,:] = mew_r  # relative permeability in part of the grid
            self.ep_r[end:self.x_steps//2,:]  = ep_r   # relative permittivity in part of the grid
            self.σ[end:self.x_steps//2,:] = σ  # electrical conductivity in part of the grid
            self.n = np.sqrt(self.mew_r*self.ep_r)
         
        return None
    
    def set_lossy_border(self, upto_pos = 30, max = 10**-2):
        self.bndry_sz = upto_pos
        grow = np.linspace(0, 1, num= upto_pos, endpoint=True)
        grow = grow**2
        for y in range(0, self.Es.shape[2], 1):                  
            self.σ[:upto_pos,y] += max*grow[::-1] 
            self.σ[-upto_pos:,y] += grow*max
        for x in range(0, self.Es.shape[1], 1):
            self.σ[x,:upto_pos] += max*grow[::-1] 
            self.σ[x,-upto_pos:] += grow*max
    
    def set_source(self, x_pos, y_pos, kind = 'Point Impusle', theta = 0):
        if kind == 'Point Impulse':
            #Gaussian source (impulse)
            #Source Parameters
            tau = 10*self.dt
            t0  = 10*tau
            
            #Generating Electric Field source over time
            Esrc = np.exp(-((self.t_axis-t0)/tau)**2) 
            self.Es[:, x_pos, y_pos] += Esrc[:]
            
        elif kind == 'Plane Impulse':
            #Gaussian source (impulse)
            #Source Parameters
            tau = 10*self.dt
            t0  = 10*tau
            #Generating Electric Field source over time
            for x in range(0, self.Es.shape[1], 1):    
                self.Es[:, x, y_pos] += 100*np.exp(-((self.t_axis-t0)/tau)**2)/self.x_axis.size
        
        elif kind == 'Point Sine':
            T = 40*self.dt
            Esrc = np.sin(2*np.pi*self.t_axis/T)
            self.Es[:, x_pos, y_pos] += Esrc[:]
        
        elif kind == 'Plane Sine Wave':
            T = 20*self.dt
            Esrc = np.sin(2*np.pi*self.t_axis/T)/self.x_axis.size
            for t in range(0, self.Es.shape[0], 1):
                self.Es[t][:,y_pos] += Esrc[t]
                
        elif kind == 'Dir Plane Sine Wave':
            m = -np.tan(theta)  #thetha is the angle made with the vertical by the wavevector
            T = 40*self.dt   # Time period of source
            Esrc = np.sin(2*np.pi*self.t_axis/T)/self.x_axis.size # Sine source
            
            for t in range(0, self.Es.shape[0], 1):
                for x in range(0, self.Es.shape[1], 1):
                    y = int(m*(x-x_pos)) + y_pos
                    if  y>= 0 and y < self.Es.shape[2]:
                        #The gau term ensures limited spread of the plane wave
                        gau = np.exp( -( ((self.x_axis[x]-self.x_axis[x_pos])/3)**2 + ((self.y_axis[y]-self.y_axis[y_pos])/3)**2 ) )
                        #Registering the source
                        self.Es[t][x,y] += 100*gau*Esrc[t]
        
        return None
import copy

def FDTD(Grid):

    dt = Grid.dt
    ds = Grid.ds
    
    mew_r = Grid.mew_r
    ep_r  = Grid.ep_r
    σ     = Grid.σ
    
    #Update Coefficients for fdtd (chosen basedon Maxwell's equations with normalized Electric and Magnetic field values)
    mH = (-dt*c0/(mew_r*ds))

    mE1 = (2*ep_0*ep_r - σ*dt)/(2*ep_0*ep_r + σ*dt)
    mE2 = 2*ep_0*c0*dt/(ds*(2*ep_0*ep_r + dt*σ))
    
    HxOverTime = np.zeros((Grid.t_steps, Grid.x_steps, Grid.y_steps)) 
    HyOverTime = np.zeros((Grid.t_steps, Grid.x_steps, Grid.y_steps)) 
    EzOverTime = np.zeros((Grid.t_steps, Grid.x_steps, Grid.y_steps)) 

    for t in range(1, Grid.t_steps, 1):
        
        Hx = copy.deepcopy(HxOverTime[t-1][:, :])
        Hy = copy.deepcopy(HyOverTime[t-1][:, :])
        Ez = copy.deepcopy(EzOverTime[t-1][:, :])

        Hx[:,:-1] += mH[:,:-1]*(Ez[:,1:]-Ez[:,:-1])  #Change in Ez over y
        Hx[:,-1] += mH[:,-1]*(0-Ez[:,-1]) #Boundary Condition (y high)

        HxOverTime[t][:,:] += Hx[:,:]

        Hy[:-1,:] -= mH[:-1,:]*(Ez[1:,:]-Ez[:-1,:])  #Change on Ez over x
        Hy[-1,:] -= mH[-1,:]*(Ez[0,:]-Ez[-1,:])  #Boundary Condition (x high)

        HyOverTime[t][:,:] += Hy[:,:]

        CHz = np.zeros(Ez.shape)  #z component of curl of H
        CHz[1:,:] += (Hy[1:,:]-Hy[:-1,:])  #Change in Hy over x
        CHz[0,:] += (Hy[0,:]- Hy[-1,:])  #Boundary Condition (x low)
        CHz[:,1:] -= (Hx[:,1:]-Hx[:,:-1])  #Change in Hx over y
        CHz[:,0] -= (Hx[:,0]-0)  #Boundary Condition (y low)
        CHz[0,0] += Hy[0,0] - Hx[0,0]  #Boundary Condition (origin)
        Ez = mE1[:,:]*Ez + mE2[:,:]*CHz[:,:]

        Ez += Grid.Es[t]  #injecting source at a point in the grid

        EzOverTime[t][:,:] += Ez[:,:]
    
    return HxOverTime, HyOverTime, EzOverTime

def perp_ref_to_ep_r(R):
    if R == 1:
        ep_r = 10**20   #arbitrarily large number to stand in for infinity
    else:
        ep_r = ((1+np.sqrt(R))/(1-np.sqrt(R)))**2
    return ep_r

def perp_loss_to_σ(L, dt, ds):    # WORK IN PROGRESS
    return ep_0*L/(105*100*dt*ds)

In [3]:
R = 0.05       #Ratio of Energy Reflected
L = 0.5         #Loss
T = 1.0 -R-L  #Ratio of Energy Tranmitted

EMGrid = Grid(t_steps = 500, x_steps = 400, y_steps = 300)  # Creating a grid to simulate over

EMGrid.set_lossy_border(40, 50*10**-(3)) # Making sure EM waves don't get to the coundary
EMGrid.set_environment_parameters(mew_r = 1.0, ep_r = perp_ref_to_ep_r(R), σ = perp_loss_to_σ(L, EMGrid.dt, EMGrid.ds), portion = 'half Y')  #Setting half the grid to have diff. dielectric constant 

EMGrid.set_source(x_pos = EMGrid.x_steps//3, y_pos = EMGrid.y_steps//3, kind = 'Point Sine', theta = 0)  # Adding directional wave source 

HxOverTime, HyOverTime, EzOverTime = FDTD(EMGrid)  # Running simulation

o2 = Figure(figsize=(7,4), dpi=80)
ax2 = o2.add_subplot(111, xlim=(0, EMGrid.y_len), ylim = (-0.05,0.05))

for i in range(0, EMGrid.y_steps, 1):
    if EMGrid.n[100,i] == 1.0:
        ax2.axvspan(i*EMGrid.ds, (i+1)*EMGrid.ds, facecolor='w')
    else:
        ax2.axvspan(i*EMGrid.ds, (i+1)*EMGrid.ds, facecolor='silver')
        
for i in range(0, EMGrid.bndry_sz, 1):
    ax2.axvspan(i*EMGrid.ds, (i+1)*EMGrid.ds, facecolor='gray')

for i in range(EMGrid.y_steps-EMGrid.bndry_sz, EMGrid.y_steps, 1):
    ax2.axvspan(i*EMGrid.ds, (i+1)*EMGrid.ds, facecolor='gray')

ax2.set_xlabel('y axis')
ax2.set_ylabel('Electric Field')
line, = ax2.plot([], [], lw=2)

def init():
    line.set_data([], [])
    return line,

#The E field over y axis is plotted every 10th time step
def animate(t):
    x = EMGrid.y_axis
    y = EzOverTime[10*t, EMGrid.x_steps//3]
    ax2.set_title("Step " + str(10*t+1) + " of " + str(EMGrid.t_steps)+ " : time = " + str(10*t*EMGrid.dt))
    line.set_data(x, y)
    return line,

        
ax2.set_xlabel('y axis')
ax2.set_ylabel('Electric Field')


Text(0, 0.5, 'Electric Field')

In [4]:
class Sphere():
    def __init__(self, *args, **kwargs):
        self.fig = Figure(figsize=(4,4), dpi=80)
        self.ax = self.fig.add_subplot(111, projection='3d')

    def draw(self, theta_num, phi_num, initview, pos_arr):
        theta_array, phi_array = np.mgrid[0.0:np.pi:theta_num*1j, 0.0:2.0*np.pi:phi_num*1j]
        x_array = np.sin(theta_array) * np.cos(phi_array)
        y_array = np.sin(theta_array) * np.sin(phi_array)
        z_array = np.cos(theta_array)
        meridian_y = np.sin(np.linspace(0, np.pi, theta_num))
        meridian_z = np.cos(np.linspace(0, np.pi, theta_num))
        meridian_x = [0]*len(meridian_y)
        self.ax.clear()
        self.ax.plot3D(meridian_x, meridian_y, meridian_z) # a reference line for rotation
        self.ax.plot_surface(x_array, y_array, z_array, alpha=0.2)
        self.ax.set_xlim([-1,1])
        self.ax.set_ylim([-1,1])
        self.ax.set_zlim([-1,1])
        self.ax.set_xlabel("Reflect amplitude")
        self.ax.set_ylabel("Transmission amplitude")
        self.ax.set_zlabel("Loss amplitude")

        self.ax.scatter(x_array, y_array, z_array, s=2)
        self.ax.scatter(pos_arr[0],pos_arr[1], pos_arr[2],s=50,color='r')
        self.ax.view_init(initview[0], initview[1])
        
class Universe(tk.Tk):
    def __init__(self, *args, **kwargs):
        tk.Tk.__init__(self, *args, **kwargs)
        tk.Tk.wm_title(self, "Welcome to Graphics")
        
        ask_theta = tk.Label(self, text="How many data points for polar(theta) angle?")
        ask_phi = tk.Label(self, text="How many data points for azimuthal(phi) angle?")
        ask_phi.grid(row=0, column=0)
        ask_theta.grid(row=1, column=0)
        
        theta_num = tk.IntVar(value=20)
        phi_num = tk.IntVar(value=20)   

        get_theta_num = tk.Entry(self, textvariable=str(theta_num))
        get_phi_num = tk.Entry(self, textvariable=str(phi_num))
        
        get_theta_num.grid(row=0, column=1)
        get_phi_num.grid(row=1, column=1)
        o1 = Sphere()
        o1.draw(int(theta_num.get()), int(phi_num.get()), [45,45], [0,0,0])
        def return_method(*args):
            nonlocal o1, canvas
            canvas.get_tk_widget().delete("all")
            o1.draw(int(theta_num.get()), int(phi_num.get()), [20,45], point_pos_arr)
            canvas = FigureCanvasTkAgg(o1.fig, self)
            canvas.get_tk_widget().grid(row=14, column=0)
            canvas._tkcanvas.grid(row=14, column=0)

        get_theta_num.bind('<Return>', return_method)
        get_phi_num.bind('<Return>', return_method) 
        
        # the sliders are from Graphics, Veronica
        self.ref_var = tk.IntVar()
        self.ref = tk.Scale(self, from_=0, to=1, orient=tk.HORIZONTAL, variable=self.ref_var, label='Reflectivity', resolution=0.01, sliderlength=10, length=150)
        self.ref.grid(row=3, column=0)
        self.ref.set(0.4)
        
        self.trans_var = tk.IntVar()
        self.trans = tk.Scale(self, from_=0, to=1, orient=tk.HORIZONTAL, variable=self.trans_var, label='Transmissivity', resolution=0.01, sliderlength=10, length=150)
        self.trans.grid(row=4, column=0)
        self.trans.set(0.6)
        
        self.loss = tk.Scale(self, from_=0, to=1, orient=tk.HORIZONTAL, label='Loss', resolution=0.01, state='disabled', sliderlength=10, length=150)
        self.loss.grid(row=5, column=0)
        def trace_method(*args):
            self.loss.config(state="normal")
            self.loss.set(1 - self.ref.get() - self.trans.get())
            self.loss.config(state="disabled")
            self.trans.config(to=1-self.ref.get()) # update to change the maximum of transmission value       
            nonlocal point_pos_arr, o1, canvas
            point_pos_arr = [round(x,3) for x in [np.sqrt(self.ref.get()), np.sqrt(self.trans.get()), np.sqrt(self.loss.get())]]   
            point_pos.config(text="The point position on RTL sphere is at " + str(point_pos_arr))
            o1.draw(int(theta_num.get()), int(phi_num.get()), [20,45], point_pos_arr)
        
        def set_values(*args):
            global R, T, L, o2, EMGrid, HxOverTime, HyOverTime, EzOverTime
            R = self.ref.get()
            T = self.trans.get()
            L = 1 - R - T
            set_values.config(text= "The point position on RTL sphere is at "+str([round(x, 3) for x in [R, T, L]]))
            EMGrid.set_environment_parameters(mew_r = 1.0, ep_r = perp_ref_to_ep_r(R), σ = perp_loss_to_σ(L, EMGrid.dt, EMGrid.ds), portion = 'half Y')  #Setting half the grid to have diff. dielectric constant 
            HxOverTime, HyOverTime, EzOverTime = FDTD(EMGrid)
            
        self.ref_var.trace("w", trace_method)
        self.trans_var.trace("w", trace_method)
        tk.Button(self, text='Set R, T, L values', command=set_values).grid(row=6,column=0)
        
        point_pos_arr = [np.sqrt(self.ref.get()), np.sqrt(self.trans.get()), np.sqrt(self.loss.get())]
        point_pos = tk.Label(self, text = "The point position on RTL sphere is at " + str(point_pos_arr))
        point_pos.grid(row=2, column=0)
        
        ask_res = tk.Label(self, text="What's the grid resolution?")
        ask_t_steps = tk.Label(self, text="What's the number of steps?")
        ask_x_steps = tk.Label(self, text="What's the number of points on x axis?")
        ask_y_steps = tk.Label(self, text="What's the number of points on y axis?")
        ask_kind = tk.Label(self, text="What wave type do you want? (Point/Plane Impusle, Point Sine, Plane Sine Wave, Dir Plane Sine Wave)")
        set_values = tk.Label(self, text="The set values for R, T, L is "+ str([self.ref.get(), self.trans.get(), self.loss.get()]))
        
        ask_res.grid(row=3, column=1)
        ask_t_steps.grid(row=4, column=1)
        ask_x_steps.grid(row=5, column=1)
        ask_y_steps.grid(row=6, column=1)
        ask_kind.grid(row=7, column=1)
        set_values.grid(row=7, column=0)
        
        res = tk.DoubleVar(value=0.1)
        t_steps = tk.IntVar(value=500)   
        x_steps = tk.IntVar(value=400)   
        y_steps = tk.IntVar(value=300)   
        kind = tk.StringVar(value="Point Sine")
        
        get_res = tk.Entry(self, textvariable=res)
        get_t_steps = tk.Entry(self, textvariable=t_steps)
        get_x_steps = tk.Entry(self, textvariable=x_steps)
        get_y_steps = tk.Entry(self, textvariable=y_steps)
        get_kind = tk.Entry(self, textvariable=kind)
        
        get_res.grid(row=3, column=2)
        get_t_steps.grid(row=4, column=2)
        get_x_steps.grid(row=5, column=2)
        get_y_steps.grid(row=6, column=2)
        get_kind.grid(row=7, column=2)
        
        def em_params_return(*args):
            global EMGrid, HxOverTime, HyOverTime, EzOverTime
            EMGrid = Grid(grid_res=float(get_res.get()), t_steps = int(get_t_steps.get()), x_steps=int(get_x_steps.get()), y_steps=int(get_y_steps.get()))
            EMGrid.set_environment_parameters(mew_r = 1.0, ep_r = perp_ref_to_ep_r(R), σ = perp_loss_to_σ(L, EMGrid.dt, EMGrid.ds), portion = 'half Y')  #Setting half the grid to have diff. dielectric constant 
            EMGrid.set_source(x_pos = EMGrid.x_steps//3, y_pos = EMGrid.y_steps//3, kind = str(get_kind.get()), theta = 0)  # Adding directional wave source 
            HxOverTime, HyOverTime, EzOverTime = FDTD(EMGrid)
            
            
        get_res.bind('<Return>', em_params_return)
        get_t_steps.bind('<Return>', em_params_return)
        get_x_steps.bind('<Return>', em_params_return)
        get_y_steps.bind('<Return>', em_params_return)
        get_kind.bind('<Return>', em_params_return)

        
        
        canvas = FigureCanvasTkAgg(o1.fig, self)
        canvas.get_tk_widget().grid(row=14, column=0)
        canvas._tkcanvas.grid(row=14, column=0)

        canvasEm = FigureCanvasTkAgg(o2, self)
        canvasEm.draw()
        canvasEm.get_tk_widget().grid(row=14, column=1)
        canvasEm._tkcanvas.grid(row=14, column=1)
    
app = Universe()
anim = FuncAnimation(o2, animate, init_func=init,
                               frames=EMGrid.t_steps//10, interval=100)
app.mainloop()